In [1]:
import pandas as pd
import pyodbc
import os
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
saved_dir = './sappm_data_generated/'
t_start0 = pd.Timestamp.now()
print(t_start0)

2021-11-12 07:16:58.449756


In [2]:
# baseurl = 'https://irpcplc.sharepoint.com'
# basesite = '/sites/IRPCApplication/IRSC/DevThermoScanAutoReport' # every share point has a home.
# siteurl = baseurl + basesite
# username = sharepoint_user
# password = 'xxxx'

# # localpath = "D:/OneDrive - IRPC/DS/Project/Upload2Sharepoint/Office365-REST-Python-Client-master/examples/data/report #123.csv"
# # remotepath = 'Shared%20Documents/SAPPM/report #123-1.csv' # existing folder path under sharepoint site.

# ctx_auth = AuthenticationContext(siteurl)
# ctx_auth.acquire_token_for_user(username, password)
# ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.

In [3]:
# # https://stackoverflow.com/questions/23696705/how-to-upload-a-file-to-sharepoint-site-using-python-script
# def upload2sharepoint():
#     with open(localpath, 'rb') as content_file:
#         file_content = content_file.read()
    
#     dir, name = os.path.split(remotepath)
#     file = ctx.web.get_folder_by_server_relative_url(dir).upload_file(name, file_content).execute_query()

In [4]:
# upload2sharepoint()

In [5]:
# localpath = "my_file.zip"
# remotepath = 'Shared%20Documents/SAPPM/my_file.zip' # existing folder path under sharepoint site.
# upload2sharepoint()

In [9]:
# https://github.com/vgrem/Office365-REST-Python-Client/blob/master/examples/sharepoint/files/upload_large_file.py
#รันได้ ใช้สำหรับไฟล์ใหญ่เกิน 250 MB
import pandas as pd
t_start=pd.Timestamp.now()
import keyring
import os
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext

sharepoint_user = keyring.get_password("sharepointonline_username", "username")
sharepoint_pass = keyring.get_password("sharepointonline_pass","password")

baseurl = 'https://irpcplc.sharepoint.com'
basesite = '/sites/IRPCWebsite/PRO/EM' # every share point has a home.
docurl = '/Shared%20Documents/SAPPM_Python_Uploaded'
siteurl = baseurl + basesite 
target_url = basesite+docurl
ctx_auth = AuthenticationContext(siteurl)
ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)
ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.

# target_url = basesite+docurl
target_folder = ctx.web.get_folder_by_server_relative_url(target_url)

size_chunk = 250*1024**2

local_path = ''

# file_size = os.path.getsize(local_path)


def print_upload_progress(offset):
    file_size = os.path.getsize(local_path)
    print("Uploaded '{}' bytes from '{}'...[{}%]".format(offset, file_size, round(offset / file_size * 100, 2)))
def upload_largefile2sharepoint(file2upload=local_path):
    t_start=pd.Timestamp.now()
    uploaded_file = target_folder.files.create_upload_session(file2upload, size_chunk, print_upload_progress).execute_query()
    print('File {0} has been uploaded successfully'.format(uploaded_file.serverRelativeUrl))
    print(f'Time2Upload {file2upload}: {pd.Timestamp.now()-t_start}')
# def print_upload_progress(offset):
#     file_size = os.path.getsize(local_path)
#     print("Uploaded '{}' bytes from '{}'...[{}%]".format(offset, file_size, round(offset / file_size * 100, 2)))
# def upload_largefile2sharepoint(file2upload=local_path):
#     t_start=pd.Timestamp.now()
#     uploaded_file = target_folder.files.create_upload_session(file2upload, size_chunk, print_upload_progress).execute_query()
#     print('File {0} has been uploaded successfully'.format(uploaded_file.serverRelativeUrl))
#     print(f'Time2Upload {file2upload}: {pd.Timestamp.now()-t_start}')   

An error occurred while retrieving token from XML response: AADSTS5000811: Unable to verify token signature. The signing key identifier does not match any valid registered keys.


ValueError: An error occurred while retrieving token from XML response: AADSTS5000811: Unable to verify token signature. The signing key identifier does not match any valid registered keys.

In [110]:
ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)

True

# <font color='red'>SAPSUMMARY</font>

In [8]:
driver = 'ODBC Driver 17 for SQL Server'
server = '10.33.18.153'
database = 'MaDB'
uid = 'maoper'
pwd = 'maoper2012'
con_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={uid};PWD={pwd}'
print(con_string)
con = pyodbc.connect(con_string)

DRIVER=ODBC Driver 17 for SQL Server;SERVER=10.33.18.153;DATABASE=MaDB;UID=maoper;PWD=maoper2012


In [10]:
#SAPSUMMARY
t_start=pd.Timestamp.now()
sql_cmd = """
SELECT  
            QMNUM AS Noti, QMTXT AS NotiDesc, RTRIM(LEFT(STRNO,4)) AS Plant4, STRNO AS FL, PLTXT AS FLDesc, QMDAT AS NotiDate, AUSVN AS MalFnSt, 
            AUSBS AS MalFnEnd, STRMN AS ReqSt, LTRMN AS ReqEn, ERDAT AS NotiCreateDate, AEDAT AS NotiChanged, 
            EQUNR AS Eq, ARBPL AS MainWC, SWERK AS MtcePlant, NAME1 AS PlantName, KOSTL AS CCTR, MSAUS AS BreakdownInd,
            [STATUS] AS NotiSysStatus, PRIOK AS NotiPri, PRIOKX AS NotiPriTxt, AUFNR AS [Order], AUART AS OrderType,
            QMART AS NotiType, EQART as FLobjType, CLASS AS FLClass, STISDD AS WorkSt, ENISDD AS WorkFn,
            DELDAT AS CancelDate, ACOST_S AS ActLaborCost, ACOST_MAT AS ActMatCost, ACOST_EXT AS ActExtCost,
            AMTAUFNR AS AmountOfWorkOrder, ISMNW AS MH, ADDDATE AS DateForModifyRecord, YEAR(erdat) as NotiCreateYear
            ,DATEDIFF(MINUTE,erdat,enisdd) as TTRMinute, [acost_s]+[acost_mat]+[acost_ext] as TotalCost,
            [acost_mat]+[acost_ext] as CashCost,convert(int,right(QMNUM,8)) as notification
FROM        SAPSUMMARY
     
"""
sapsummary = pd.read_sql(sql_cmd,con)
sapsummary.drop_duplicates(subset='Noti',keep='first',inplace=True)
print(f'Time2Run SapSummary{pd.Timestamp.now()-t_start}')
#sapsummary

Time2Run SapSummary0 days 00:05:29.263816


In [11]:
# t_start=pd.Timestamp.now()
# sapsummary.to_csv('sapsummary.csv')
# print(f'Time2Save SapSummary.csv{pd.Timestamp.now()-t_start}')

In [12]:
# upload_largefile2sharepoint(file2upload='sapsummary.csv')

In [13]:
# localpath = "sapsummary.csv"
# remotepath = 'Shared%20Documents/SAPPM/sapsummary.csv' # existing folder path under sharepoint site.
# upload2sharepoint()

In [14]:
# localpath = "my_file.zip"
# remotepath = 'Shared%20Documents/SAPPM/my_file.zip' # existing folder path under sharepoint site.
# upload2sharepoint()

In [15]:
# len(sapsummary)

In [16]:
#sapsummary[sapsummary.duplicated(subset='Noti',keep=False)]

In [17]:
#SAPORDER
t_start=pd.Timestamp.now()
sql_cmd = """
WITH
     b1 AS (
            SELECT AUFNR as [Order_Order], max(ADDDATE) AS Order_AddDate
            FROM SAPWORKORDER
            GROUP BY AUFNR),
     b AS (
            Select 
                AUFNR as [Order_Order],KTEXT AS [OrderDesc] , ILART AS [MainActType] , ERDAT AS [OrderCreateDate] , 
                IDAT2 AS [TecoDate], STATUS AS [OrderSysStatus] , MAUFNR AS [Superior Order], ADDDATE, 
                userstatus AS [OrderUserStatus], GSTRP AS [OrderBasicStart], GLTRP AS [OrderBasicFinish]
            from SAPWORKORDER x

            inner join b1 on x.AUFNR=b1.Order_Order and x.ADDDATE=b1.Order_AddDate
            Group by AUFNR,KTEXT,ILART,ERDAT,IDAT2,STATUS,MAUFNR,ADDDATE,userstatus,GSTRP,GLTRP)

SELECT *
FROM b
     
"""
saporder = pd.read_sql(sql_cmd,con)
saporder.drop_duplicates(subset='Order_Order',keep='first',inplace=True)
print(f'Time2Run SapOrder{pd.Timestamp.now()-t_start}')
#sapsummary

Time2Run SapOrder0 days 00:02:10.589530


In [18]:
saporder.columns

Index(['Order_Order', 'OrderDesc', 'MainActType', 'OrderCreateDate',
       'TecoDate', 'OrderSysStatus', 'Superior Order', 'ADDDATE',
       'OrderUserStatus', 'OrderBasicStart', 'OrderBasicFinish'],
      dtype='object')

In [19]:
#saporder[saporder.duplicated(subset='Order_Order',keep=False)]

In [20]:
# enfo_saporder
t_start=pd.Timestamp.now()
sql_cmd = """
SELECT  
            AUFNR AS [Order_Order], KTEXT AS [Order_Desc], VAPLZ AS [Order_MainWC], AUART AS [Order_Type], ILART AS [Order_MainActType], 
            ERDAT AS [Order_CreateDate], EQUNR AS [Order_Eq], STRNO AS [Order_FL], QMNUM AS [Order_Noti], ACOST_S AS [Order_IRPCMHCost], 
            ACOST_MAT AS [Order_MatCost], ACOST_SERL AS [Order_LaborCost], ACOST_SERM AS [Order_ServiceCost], ACOST_SERQ AS [Order_ServiceEqCost], 
            STISDD AS [Order_Start], ENISDD AS [Order_Finish], ISMNW AS [Order_MH], STATUS AS [Order_SysStatus], MAUFNR AS [Order_SupOrder], 
            userstatus AS [Order_UserStatus], IDAT2 AS [Order_TecoDate]

FROM        SAPWORKORDER
WHERE       year(ERDAT)>=2018 and AUART < 'PM09'
     
"""
enfo_saporder = pd.read_sql(sql_cmd,con)
enfo_saporder.drop_duplicates(subset='Order_Order',keep='first',inplace=True)
enfo_saporder['Order_TotalCost'] = enfo_saporder.Order_LaborCost+enfo_saporder.Order_MatCost+enfo_saporder.Order_IRPCMHCost+enfo_saporder.Order_ServiceCost+enfo_saporder.Order_ServiceEqCost
enfo_saporder['Order_CashCost'] = enfo_saporder.Order_LaborCost+enfo_saporder.Order_MatCost+enfo_saporder.Order_ServiceCost+enfo_saporder.Order_ServiceEqCost
print(f'Time2Run enfo_saporder{pd.Timestamp.now()-t_start}')
#sapsummary

Time2Run enfo_saporder0 days 00:01:43.929962


In [21]:
#SAPNOTIFICATION
t_start=pd.Timestamp.now()
sql_cmd = """
SELECT 
        QMNUM AS Noti_noti,INGRP as plannergroup, QMGRP as failuremode_codegroup, QMCOD as failuemode_id, 
		QMCOD_D as failuremode, userstatus as NotiUserStatus, notxt as notilongtxt, inspk as ref_pm_order,
        ROOMNUMBER1 as NotiCreateByID, ROOMNUMBER2 as NotiCompleteByID
FROM    SAPNOTIFICATION
"""
sapnotification = pd.read_sql(sql_cmd,con)
sapnotification.drop_duplicates(subset='Noti_noti',keep='first',inplace=True)
print(f'Time2Run SapNotification{pd.Timestamp.now()-t_start}')

Time2Run SapNotification0 days 00:00:47.075205


In [22]:
sapnotification

,Noti_noti,plannergroup,failuremode_codegroup,failuemode_id,failuremode,NotiUserStatus,notilongtxt,ref_pm_order,NotiCreateByID,NotiCompleteByID
0,000010232822,S14,,,None,None,None,None,None,None
1,000010232823,S16,,,None,None,None,None,None,None
2,000020712794,S15,,,None,None,None,None,None,None
3,000020712795,S15,,,None,None,None,None,None,None
4,000070041543,S13,,,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
2379840,000010727223,RRC,,,None,APPR,ไฟส่อง LG ของ boot น้ำ 52D011 ขาด,,00005238,00000670
2379841,000010727229,PP2,,,None,APPR,,,00005066,00005094
2379842,000010727230,PP2,,,None,APPR,,,00004460,00004476
2379843,000010727231,RLB,,,None,APPR,Renew ใบพัด 03E012Lเนื่องจากการปรับปรุงเรื่องก...,,00004160,00004116


In [23]:
#employee code
#t_start=pd.Timestamp.now()
#sql_cmd = """
#SELECT 
#        *
#FROM    master_employee
#"""
#employee_master = pd.read_sql(sql_cmd,con)
##sapnotification.drop_duplicates(subset='Noti_noti',keep='first',inplace=True)
#print(f'Time2Run employee master{pd.Timestamp.now()-t_start}')

In [24]:
#employee_code = employee_master[['employee_id','name_surname_th','section_en']]

In [25]:
#sapnotification = sapnotification.merge(employee_code,left_on='NotiCreateByID',right_on='employee_id',how='left')
#sapnotification.rename(columns={'name_surname_th':'NotiCreateBy','section_en':'NotiCreateSec'},inplace=True)
#sapnotification = sapnotification.merge(employee_code,left_on='NotiCompleteByID',right_on='employee_id',how='left')
#sapnotification.rename(columns={'name_surname_th':'NotiCompleteBy','section_en':'NotiCompleteSec'},inplace=True)

In [26]:
driver = 'ODBC Driver 17 for SQL Server'
server = '10.31.1.41'
database = 'Intranet'
uid = 'ma_reader'
pwd = 'passw0rd@1'
con_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={uid};PWD={pwd}'
print(con_string)
con = pyodbc.connect(con_string)

DRIVER=ODBC Driver 17 for SQL Server;SERVER=10.31.1.41;DATABASE=Intranet;UID=ma_reader;PWD=passw0rd@1


In [27]:
#employee code
t_start=pd.Timestamp.now()
sql_cmd = """
SELECT 
        *
FROM    vw_getEmployee
"""
employee = pd.read_sql(sql_cmd,con)
#sapnotification.drop_duplicates(subset='Noti_noti',keep='first',inplace=True)
print(f'Time2Run employee master{pd.Timestamp.now()-t_start}')

Time2Run employee master0 days 00:00:00.896024


In [28]:
employee_code = employee[['EmpId','THName','OrgShortD']]

In [29]:
employee_code.tail(3)

,EmpId,THName,OrgShortD
7424,00008415,นาย วีระชัย วงค์ตะลา,DIV PWRD
7425,00008421,น.ส. วันทนีย์ ทิพย์สุนาวี,DEP CSSI
7426,00008423,MR. Nobuyuki Nakashima,GRO CCCM


In [30]:
sapnotification = sapnotification.merge(employee_code,left_on='NotiCreateByID',right_on='EmpId',how='left')
sapnotification.rename(columns={'THName':'NotiCreateBy','OrgShortD':'NotiCreateSec'},inplace=True)
sapnotification = sapnotification.merge(employee_code,left_on='NotiCompleteByID',right_on='EmpId',how='left')
sapnotification.rename(columns={'THName':'NotiCompleteBy','OrgShortD':'NotiCompleteSec'},inplace=True)

In [31]:
sapnotification.columns

Index(['Noti_noti', 'plannergroup', 'failuremode_codegroup', 'failuemode_id',
       'failuremode', 'NotiUserStatus', 'notilongtxt', 'ref_pm_order',
       'NotiCreateByID', 'NotiCompleteByID', 'EmpId_x', 'NotiCreateBy',
       'NotiCreateSec', 'EmpId_y', 'NotiCompleteBy', 'NotiCompleteSec'],
      dtype='object')

In [32]:
#sapnotification[sapnotification.duplicated(subset='Noti_noti',keep=False)]

In [33]:
t_start = pd.Timestamp.now()
sapsummary = sapsummary.merge(sapnotification,left_on='Noti',right_on='Noti_noti',how='left')
print(f'Time2Merge sapnotification = {pd.Timestamp.now()-t_start}')

Time2Merge sapnotification = 0 days 00:00:13.990998


In [34]:
#sapsummary[sapsummary.duplicated(subset='Noti',keep=False)]

In [35]:
t_start = pd.Timestamp.now()
sapsummary = sapsummary.merge(saporder,left_on='Order',right_on='Order_Order',how='left')
print(f'Time2Merge order = {pd.Timestamp.now()-t_start}')

Time2Merge order = 0 days 00:00:15.783998


In [36]:
#sapsummary[sapsummary.duplicated(subset='Noti',keep=False)]

# <font color='red'>PMWH</font>

In [37]:
t1_start = pd.Timestamp.now()
driver = 'ODBC Driver 17 for SQL Server'
server = '10.31.1.72'
# database = 'Demo'
database = 'PMWH'
uid = 'pm_reader'
pwd = 'pmreader@2020'
con_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={uid};PWD={pwd}'
print(con_string)
con = pyodbc.connect(con_string)
print(f'Time to call FLEQ server = {pd.Timestamp.now()-t1_start}')

DRIVER=ODBC Driver 17 for SQL Server;SERVER=10.31.1.72;DATABASE=PMWH;UID=pm_reader;PWD=pmreader@2020
Time to call FLEQ server = 0 days 00:00:00.267006


## <font color='blue'>Functional Location Data</font>

In [38]:
# get data from MaDB SAPSUMMARY
t1_start = pd.Timestamp.now()

sql_cmd = """
    select  [TPLNR] AS [flid], [STRNO] AS [fl], [TPLKZ] AS [flstrid], [FLTYP] AS [mei], [PLTXT] AS [fltxt],
    [TPLMA] AS [supflid], [STRNO_SUP] AS [supfl], [EQART] AS [fltype], [OBJNR] AS [Object number],
    [HERST] AS [Manufacturer of asset], [SERGE] AS [Manufacturer serial number], [TYPB] AS [Manufacturer model number],
    [POSNR] AS [Position in superior technical object], [ABCKZ] AS [FLabc], [SWERK] AS [Maintenance plant],
    [IWERK] AS [Maintenance Planning Plant], [INGRP] AS [Planner Group], [GEWRK] AS [Maintenance Work Center ID],
    [workcenterName] AS [Maintenance Work Center Name], [RBNR] AS [Catalog Profile], [GSBER] AS [Business Area],
    [KOKRS] AS [Controlling Area], [KOSTL] AS [Cost Center], [BUKRS] AS [Company Code],
    [STAT] AS [flstatus], [AEDAT] AS [flchange]


        from dbo.vw_func 

"""
fldf = pd.read_sql(sql_cmd,con)
fldf.drop_duplicates(subset='fl',keep='first',inplace=True)
print(f'Time2Run fldf = {pd.Timestamp.now()-t1_start}')

Time2Run fldf = 0 days 00:00:10.707972


In [39]:
t1_start = pd.Timestamp.now()
fldf.to_excel(saved_dir+'fldf.xlsx',encoding='UTF-8',index=False)
print(f'Time2Save eqdf = {pd.Timestamp.now()-t1_start}')

Time2Save eqdf = 0 days 00:01:56.678783


In [40]:
local_path = saved_dir+'fldf.xlsx'
upload_largefile2sharepoint(local_path)

NameError: name 'upload_largefile2sharepoint' is not defined

In [ ]:
#fldf.duplicated(keep=False).sum()

## <font color='blue'>Equipment Data</font>

In [41]:
# get data from PMWH Equipment data
t1_start = pd.Timestamp.now()

sql_cmd = """
    select  [EQUNR] AS [eq], [EQKTX] AS [eqtxt], [EQTYP] AS [mei], [EQART] AS [eqtype], [OBJNR] AS [Object number],
    [HERST] AS [Manufacturer of asset], [SERGE] AS [Manufacturer serial number], [TYPB] AS [Manufacturer model number],
    [ABCKZ] AS [Eqabc], [SWERK] AS [Maintenance plant], [IWERK] AS [Maintenance Planning Plant], [INGRP] AS [Planner Group],
    [GEWRK] AS [mainwcid], [workcenterName] AS [mainwc], [RBNR] AS [catalog], [GSBER] AS [Business Area],
    [KOKRS] AS [Controlling Area], [KOSTL] AS [Cost Center], [BUKRS] AS [Company Code], [STAT] AS [eqstatus],
    [AEDAT] AS [eqchangedate], [TPLNR] AS [fl], [classDescr] AS eqclass


    from dbo.vw_equipment a
	left join vw_equipment_class b
	on a.EQUNR=b.equipment

"""
eqdf = pd.read_sql(sql_cmd,con)
eqdf.drop_duplicates(subset='eq',keep='first',inplace=True)
print(f'Time2Run eqdf = {pd.Timestamp.now()-t1_start}')

Time2Run eqdf = 0 days 00:00:11.582545


In [42]:
t1_start = pd.Timestamp.now()
eqdf.to_excel(saved_dir+'eqdf.xlsx',encoding='UTF-8',index=False)
print(f'Time2Save eqdf = {pd.Timestamp.now()-t1_start}')

Time2Save eqdf = 0 days 00:02:01.186082


In [43]:
local_path = saved_dir+'eqdf.xlsx'
upload_largefile2sharepoint(local_path)

NameError: name 'upload_largefile2sharepoint' is not defined

In [ ]:
#eqdf.duplicated(keep=False).sum()

## <font color='blue'>IW22 Item Tab</font>

In [44]:
# get data from PMWH Tab item data
t1_start = pd.Timestamp.now()

sql_cmd = """
    select  QMNUM as noti, FENUM as itemnr, FETXT as notitxt, FEKAT as cattype_failuremech, FEGRP as failuremech_codegroup,
    FEGRPD as failuremech_codegroupdesc, FECOD as failuermech_id, FECODD as failuremech, OTGRP mtceitem_codegroup,
    OTGRPD as mtceitem_subunit, OTCOD as mtceitem_id, OTCODD as mtceitem_part, changeDate as notichangedate

    from dbo.vw_qmfe 
    where fenum = 1

"""
fme_part= pd.read_sql(sql_cmd,con)
fme_part.drop_duplicates(subset='noti',keep='first',inplace=True)

print(f'Time2Run fme_part = {pd.Timestamp.now()-t1_start}')

Time2Run fme_part = 0 days 00:00:02.870644


In [45]:
#len(fme_part[fme_part.duplicated(subset='noti',keep=False)])

## <font color='blue'>IW22 Item >> Maintenane Activity Tab</font>

In [46]:
# get data from PMWH Item Tab >> Maintenance item data
t1_start = pd.Timestamp.now()

sql_cmd = """
    select  [QMNUM] AS [noti], FENUM AS [itemnr], MATXT AS [mtceact_txt], MNKAT AS [catttype_mtceact],
    MNGRP AS [mtceact_codegroup], MNGRPD AS [mtceact_codegroupdesc], MNCOD AS [mtceact_id], MNCODD AS [mtceact],
    changedate as notichangedate


    from dbo.vw_qmma
    where fenum = 1

"""
mtceact = pd.read_sql(sql_cmd,con)
mtceact.drop_duplicates(subset='noti',keep='first',inplace=True)


print(f'Time2Run mtceact = {pd.Timestamp.now()-t1_start}')

Time2Run mtceact = 0 days 00:00:01.455397


In [47]:
#mtceact.duplicated(subset='noti',keep=False).sum()

## <font color='blue'>IW22 Item Tab >> Cause Tab</font>

In [48]:
# get data from PMWH Tab item >> Cause Tab data
t1_start = pd.Timestamp.now()

sql_cmd = """
    select  qmnum as noti, fenum as itemnr, urtxt as causetxt, urgrp as cause_codegroup,
    urgrpd as cause_codegroupdesc,urcod as cause_id, urcodd as cause, changedate as notichangedate


    from dbo.vw_qmur
    where fenum = 1

"""
cause = pd.read_sql(sql_cmd,con)
cause.drop_duplicates(subset='noti',keep='first',inplace=True)

print(f'Time2Run cause = {pd.Timestamp.now()-t1_start}')

Time2Run cause = 0 days 00:00:01.541190


In [49]:
#cause.duplicated(subset='noti',keep=False).sum()

# Merge FL Type and EQ Type

In [50]:
t1_start = pd.Timestamp.now()
sapsummary = sapsummary.merge(fldf, left_on='FL', right_on='fl',how='left')
sapsummary = sapsummary.merge(eqdf, left_on='Eq', right_on='eq',how='left')
print(f'Time2Merge SapSummary-FL-EQ = {pd.Timestamp.now()-t1_start}')

Time2Merge SapSummary-FL-EQ = 0 days 00:00:20.061021


In [51]:
#sapsummary[sapsummary.duplicated(subset='Noti',keep=False)]

In [52]:
colls = ['Plant4', 'Noti', 'NotiDesc','notilongtxt', 'ref_pm_order', 'FL','fltype', 'FLabc', 'FLDesc', 'flstatus', 'NotiDate', 'MalFnSt',
       'MalFnEnd', 'ReqSt', 'ReqEn', 'NotiCreateDate', 'NotiChanged', 'Eq','eqtype','eqclass', 'Eqabc', 'eqstatus',
       'MainWC', 'MtcePlant', 'PlantName', 'CCTR', 'BreakdownInd',
       'NotiSysStatus','NotiUserStatus', 'NotiPri', 'NotiPriTxt', 'Order', 'OrderType',
       'NotiType', 'FLobjType', 'FLClass', 'WorkSt', 'WorkFn','OrderBasicStart','OrderBasicFinish', 'CancelDate',
       'ActLaborCost', 'ActMatCost', 'ActExtCost', 'AmountOfWorkOrder', 'MH',
       'DateForModifyRecord', 'NotiCreateYear', 'TTRMinute', 'TotalCost',
       'CashCost', 'notification', 'Order_Order', 'OrderDesc', 'MainActType',
       'OrderCreateDate', 'TecoDate', 'OrderSysStatus','OrderUserStatus', 'Superior Order',
       'ADDDATE' , 'plannergroup','failuremode_codegroup', 'failuemode_id', 'failuremode',
       'NotiCreateByID', 'NotiCreateBy','NotiCreateSec','NotiCompleteByID', 'NotiCompleteBy', 'NotiCompleteSec'
       ]

In [53]:
t1_start = pd.Timestamp.now()
sapsummary = sapsummary[colls]
print(f'Time2SelectColumns FLEQ = {pd.Timestamp.now()-t1_start}')

Time2SelectColumns FLEQ = 0 days 00:00:38.893255


# Merge Failure Mechanism and Part

In [54]:
t1_start = pd.Timestamp.now()
sapsummary = sapsummary.merge(fme_part,left_on='Noti',right_on='noti',how='left',suffixes=('','_1'))
print(f'Time2Merge Faiure Mechanism and Part = {pd.Timestamp.now()-t1_start}')

Time2Merge Faiure Mechanism and Part = 0 days 00:01:13.392998


In [55]:
#sapsummary.duplicated(subset='Noti',keep=False).sum()

In [56]:
colls = ['Plant4', 'Noti', 'NotiDesc','notilongtxt', 'ref_pm_order', 'FL', 'fltype', 'FLabc',
       'FLDesc', 'flstatus', 'NotiDate', 'MalFnSt', 'MalFnEnd', 'ReqSt',
       'ReqEn', 'NotiCreateDate', 'NotiChanged', 'Eq', 'eqtype','eqclass', 'Eqabc',
       'eqstatus', 'MainWC', 'MtcePlant', 'PlantName', 'CCTR', 'BreakdownInd',
       'NotiSysStatus','NotiUserStatus', 'NotiPri', 'NotiPriTxt', 'Order', 'OrderType',
       'NotiType', 'FLobjType', 'FLClass', 'WorkSt', 'WorkFn','OrderBasicStart','OrderBasicFinish', 'CancelDate',
       'ActLaborCost', 'ActMatCost', 'ActExtCost', 'AmountOfWorkOrder', 'MH',
       'DateForModifyRecord', 'NotiCreateYear', 'TTRMinute', 'TotalCost',
       'CashCost', 'notification', 'Order_Order', 'OrderDesc', 'MainActType',
       'OrderCreateDate', 'TecoDate', 'OrderSysStatus','OrderUserStatus', 'Superior Order',
       'ADDDATE', 'plannergroup', 'failuremode_codegroup', 'failuemode_id',
       'failuremode', 'cattype_failuremech', 'failuremech_codegroup', 'failuremech_codegroupdesc', 'failuermech_id',
       'failuremech', 'mtceitem_codegroup', 'mtceitem_subunit', 'mtceitem_id', 'mtceitem_part',
        'NotiCreateByID', 'NotiCreateBy','NotiCreateSec','NotiCompleteByID', 'NotiCompleteBy', 'NotiCompleteSec'
        ]

In [57]:
t1_start = pd.Timestamp.now()
sapsummary = sapsummary[colls]
print(f'Time2SelectColumns Failure Mechanism and Part = {pd.Timestamp.now()-t1_start}')

Time2SelectColumns Failure Mechanism and Part = 0 days 00:00:07.646729


# Merge Maintenace Activity

In [58]:
t1_start = pd.Timestamp.now()
sapsummary = sapsummary.merge(mtceact,left_on='Noti',right_on='noti',how='left')
print(f'Time2Merge Maintenance Activity = {pd.Timestamp.now()-t1_start}')

Time2Merge Maintenance Activity = 0 days 00:00:42.129405


In [59]:
#sapsummary.duplicated(subset='Noti',keep=False).sum()

In [60]:
colls = ['Plant4', 'Noti', 'NotiDesc','notilongtxt', 'ref_pm_order', 'FL', 'fltype', 'FLabc',
       'FLDesc', 'flstatus', 'NotiDate', 'MalFnSt', 'MalFnEnd', 'ReqSt',
       'ReqEn', 'NotiCreateDate', 'NotiChanged', 'Eq', 'eqtype', 'eqclass', 'Eqabc',
       'eqstatus', 'MainWC', 'MtcePlant', 'PlantName', 'CCTR', 'BreakdownInd',
       'NotiSysStatus','NotiUserStatus', 'NotiPri', 'NotiPriTxt', 'Order', 'OrderType',
       'NotiType', 'FLobjType', 'FLClass', 'WorkSt', 'WorkFn','OrderBasicStart','OrderBasicFinish', 'CancelDate',
       'ActLaborCost', 'ActMatCost', 'ActExtCost', 'AmountOfWorkOrder', 'MH',
       'DateForModifyRecord', 'NotiCreateYear', 'TTRMinute', 'TotalCost',
       'CashCost', 'notification', 'Order_Order', 'OrderDesc', 'MainActType',
       'OrderCreateDate', 'TecoDate', 'OrderSysStatus', 'OrderUserStatus','Superior Order',
       'ADDDATE', 'plannergroup', 'failuremode_codegroup', 'failuemode_id',
       'failuremode', 'cattype_failuremech', 'failuremech_codegroup',
       'failuremech_codegroupdesc', 'failuermech_id', 'failuremech',
       'mtceitem_codegroup', 'mtceitem_subunit', 'mtceitem_id',
       'mtceitem_part', 'catttype_mtceact',
       'mtceact_codegroup', 'mtceact_codegroupdesc', 'mtceact_id', 'mtceact',
        'NotiCreateByID', 'NotiCreateBy','NotiCreateSec','NotiCompleteByID', 'NotiCompleteBy', 'NotiCompleteSec']

In [61]:
t1_start = pd.Timestamp.now()
sapsummary = sapsummary[colls]
print(f'Time2SelectColumns Maintenance Activity = {pd.Timestamp.now()-t1_start}')

Time2SelectColumns Maintenance Activity = 0 days 00:00:07.651372


# Merge Root Cause

In [62]:
t1_start = pd.Timestamp.now()
sapsummary = sapsummary.merge(cause,left_on='Noti',right_on='noti',how='left')
print(f'Time2Merge Root Cause = {pd.Timestamp.now()-t1_start}')

Time2Merge Root Cause = 0 days 00:00:32.148050


In [63]:
#sapsummary.duplicated(subset='Noti',keep=False).sum()

In [64]:
colls = ['Plant4', 'Noti', 'NotiDesc', 'notilongtxt', 'ref_pm_order','FL', 'fltype', 'FLabc',
       'FLDesc', 'flstatus', 'NotiDate', 'MalFnSt', 'MalFnEnd', 'ReqSt',
       'ReqEn', 'NotiCreateDate', 'NotiChanged', 'Eq', 'eqtype', 'eqclass', 'Eqabc',
       'eqstatus', 'MainWC', 'MtcePlant', 'PlantName', 'CCTR', 'BreakdownInd',
       'NotiSysStatus','NotiUserStatus', 'NotiPri', 'NotiPriTxt', 'Order', 'OrderDesc', 'OrderType',
       'NotiType', 'FLobjType', 'FLClass', 'WorkSt', 'WorkFn','OrderBasicStart','OrderBasicFinish',
       'ActLaborCost', 'ActMatCost', 'ActExtCost', 'AmountOfWorkOrder', 'MH',
       'NotiCreateYear', 'TTRMinute', 'TotalCost',
       'CashCost', 'notification', 'MainActType',
       'OrderCreateDate', 'TecoDate', 'OrderSysStatus','OrderUserStatus', 'Superior Order',
       'plannergroup', 'failuremode_codegroup', 'failuemode_id',
       'failuremode', 'cattype_failuremech', 'failuremech_codegroup',
       'failuremech_codegroupdesc', 'failuermech_id', 'failuremech',
       'mtceitem_codegroup', 'mtceitem_subunit', 'mtceitem_id',
       'mtceitem_part', 'catttype_mtceact', 'mtceact_codegroup',
       'mtceact_codegroupdesc', 'mtceact_id', 'mtceact',
       'causetxt', 'cause_codegroup', 'cause_codegroupdesc', 'cause_id',
       'cause',
         'NotiCreateByID', 'NotiCreateBy','NotiCreateSec','NotiCompleteByID', 'NotiCompleteBy', 'NotiCompleteSec']

In [65]:
t1_start = pd.Timestamp.now()
sapsummary = sapsummary[colls]
print(f'Time2SelectColumns Root Cause = {pd.Timestamp.now()-t1_start}')

Time2SelectColumns Root Cause = 0 days 00:00:07.789029


In [66]:
#sapsummary.duplicated(subset='Noti',keep=False).sum()

In [67]:
#sapsummary[sapsummary.duplicated(subset='Noti',keep=False)]

In [68]:
sapsummary.columns

Index(['Plant4', 'Noti', 'NotiDesc', 'notilongtxt', 'ref_pm_order', 'FL',
       'fltype', 'FLabc', 'FLDesc', 'flstatus', 'NotiDate', 'MalFnSt',
       'MalFnEnd', 'ReqSt', 'ReqEn', 'NotiCreateDate', 'NotiChanged', 'Eq',
       'eqtype', 'eqclass', 'Eqabc', 'eqstatus', 'MainWC', 'MtcePlant',
       'PlantName', 'CCTR', 'BreakdownInd', 'NotiSysStatus', 'NotiUserStatus',
       'NotiPri', 'NotiPriTxt', 'Order', 'OrderDesc', 'OrderType', 'NotiType',
       'FLobjType', 'FLClass', 'WorkSt', 'WorkFn', 'OrderBasicStart',
       'OrderBasicFinish', 'ActLaborCost', 'ActMatCost', 'ActExtCost',
       'AmountOfWorkOrder', 'MH', 'NotiCreateYear', 'TTRMinute', 'TotalCost',
       'CashCost', 'notification', 'MainActType', 'OrderCreateDate',
       'TecoDate', 'OrderSysStatus', 'OrderUserStatus', 'Superior Order',
       'plannergroup', 'failuremode_codegroup', 'failuemode_id', 'failuremode',
       'cattype_failuremech', 'failuremech_codegroup',
       'failuremech_codegroupdesc', 'failuermech_id'

In [69]:
#complex = pd.read_excel('N:/OneDrive - IRPC/ComplexPlantCalendar/DateCalendar.xlsx',sheet_name='ComplexPlant')
complex = pd.read_excel('DateCalendar.xlsx',sheet_name='ComplexPlant')
pgdiv = pd.read_excel('DateCalendar.xlsx',sheet_name='PGDiv')

In [70]:
eqpop = eqdf[['eq','eqtype','eqclass']][eqdf.eqstatus=='ACTIVE'].copy()
eqpop.loc[:,'plant'] = eqpop['eq'].str[:4]
eqpop.loc[:,'plant'] = eqpop.plant.apply(lambda v: v.strip())
eqpop.loc[:,'plant'] = eqpop.plant.apply(lambda v: v.replace('-',''))

In [71]:
eqpop = eqpop.merge(complex, left_on='plant',right_on='Plant',how='left')
eqpop = eqpop[['eq','eqtype','eqclass','plant','Complex']]
eqpop.rename(columns={'Complex':'complex'},inplace=True)

In [72]:
eqpop = eqpop.pivot_table(index=['complex','plant','eqtype','eqclass'],aggfunc='count').reset_index()
eqpop.rename(columns={'eq':'population'},inplace=True)

In [73]:
#eqpop.groupby(by=['complex','plant','eqtype']).agg(pop=('eqclass','count'))

In [74]:
# eqpop.to_excel('eqpop.xlsx',index=False)

In [75]:
eqpop.to_excel(saved_dir+'eqpop.xlsx',index=False)

In [76]:
local_path = saved_dir+'eqpop.xlsx'
upload_largefile2sharepoint(local_path)

NameError: name 'upload_largefile2sharepoint' is not defined

In [77]:
sapsummary = sapsummary.merge(complex,left_on='Plant4',right_on='Plant',how='left')

In [78]:
# get a list of columns
cols = list(sapsummary)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Complex')))
cols.insert(1, cols.pop(cols.index('Plant')))

In [79]:
sapsummary = sapsummary[cols]

In [80]:
#sapsummary[['Complex','Plant','Eq','eqtype','Noti','eqclass']].merge(eqpop[['plant','eqtype','eqclass','population']].groupby(['plant','eqtype','eqclass']).sum().reset_index(),left_on=['Plant','eqtype','eqclass'],right_on=['plant','eqtype','eqclass'],how='left')

In [81]:
sapsummary = sapsummary.merge(eqpop,left_on=['Plant','eqtype','eqclass'],right_on=['plant','eqtype','eqclass'],how='left')
sapsummary['population'].fillna(0,inplace=True)

In [82]:
#sapsummary.loc[3:4][['OrderUserStatus','OrderSysStatus']]

In [83]:
#sapsummary[(sapsummary.OrderUserStatus.str.contains('FIN')==True)|(sapsummary.OrderSysStatus.str.contains('CLSD|TECO')==True)][['OrderUserStatus','OrderSysStatus']].head(50)

In [84]:
sapsummary['finished'] = (sapsummary.OrderUserStatus.str.contains('FIN')==True)|(sapsummary.OrderSysStatus.str.contains('CLSD|TECO')==True)

In [85]:
#sapsummary[(sapsummary.NotiSysStatus.str.contains('DLFL')==True)|(sapsummary.OrderUserStatus.str.contains('CAN'))][['NotiSysStatus','OrderUserStatus','OrderSysStatus']]

In [86]:
sapsummary['canceled'] = (sapsummary.NotiSysStatus.str.contains('DLFL')==True)|(sapsummary.OrderUserStatus.str.contains('CAN')==True)

In [87]:
#sapsummary[(~sapsummary.MalFnSt.isnull())&(~sapsummary.MalFnEnd.isnull())&(sapsummary.NotiUserStatus.str.contains('SRC')==True)&(sapsummary.NotiUserStatus.str.contains('FCC')==True)][['NotiUserStatus','MalFnSt','MalFnEnd']]

In [88]:
sapsummary['dqp'] = (~sapsummary.MalFnSt.isnull())&(~sapsummary.MalFnEnd.isnull())&(sapsummary.NotiUserStatus.str.contains('SRC')==True)&(sapsummary.NotiUserStatus.str.contains('FCC')==True)

In [89]:
#sapsummary[(sapsummary.dqp==False)&(sapsummary.finished)][['dqp','finished']]

In [90]:
sapsummary['needhist'] = (sapsummary.dqp==False)&(sapsummary.finished==True)&(sapsummary.canceled==False)&(sapsummary.NotiType=='M1')&(sapsummary.OrderType=='PM01')
sapsummary['FCC'] = sapsummary.NotiUserStatus.str.contains('FCC')
sapsummary['SRC'] = sapsummary.NotiUserStatus.str.contains('SRC')

In [91]:
# sapsummary[sapsummary.NotiUserStatus.str.contains('FCC')==True]

In [92]:
sapsummary = sapsummary.merge(pgdiv,on='plannergroup',how='left')

In [93]:
# sapsummary = pd.read_csv('sappm.csv',low_memory=False)

In [94]:
# sapsummary.columns

In [95]:
# gen excel for Maintenance Failure Recording Data Quality Check
t1_start = pd.Timestamp.now()
sapsummary[(sapsummary.needhist==True)&(sapsummary.NotiCreateDate>='2019-06-01')&(sapsummary.OrderType=='PM01')&(sapsummary.NotiType=='M1')]\
        [['MalFnSt', 'MalFnEnd', 'SRC','FCC','Noti','Order', 'NotiDesc', 'notilongtxt',
        'FL', 'Eq' , 'NotiCreateDate',
        'Div','plannergroup','MainWC','NotiSysStatus','OrderType',
        'failuremode', 'failuremech', 'cause', 'mtceact','mtceitem_subunit', 'mtceitem_part',
       'NotiType', 'FLobjType', 'FLClass', 'WorkSt', 'WorkFn',                                                                                                                                       
       'NotiCreateByID', 'NotiCreateBy', 'NotiCreateSec', 'NotiCompleteByID',
       'NotiCompleteBy', 'NotiCompleteSec', 'NotiUserStatus',  'complex', 'plant',
       'finished', 'canceled', 'dqp', 'needhist',  'Div']].to_excel(saved_dir+'ForMa.xlsx',index=False)
print(f'Time2Save ForMa Excel File = {pd.Timestamp.now()-t1_start}')

Time2Save ForMa Excel File = 0 days 00:00:18.321663


In [96]:
local_path = saved_dir+'ForMa.xlsx'
upload_largefile2sharepoint(file2upload=local_path)

NameError: name 'upload_largefile2sharepoint' is not defined

In [97]:
# gen excel Data Quality Passed for Maintenance Failure Recording Data Quality Check
t1_start = pd.Timestamp.now()
sapsummary[(sapsummary['dqp']==True)&(sapsummary.NotiCreateDate>='2019-06-01')&(sapsummary.OrderType=='PM01')&(sapsummary.NotiType=='M1')]\
        [['MalFnSt', 'MalFnEnd', 'SRC','FCC','Noti','Order', 'NotiDesc', 'notilongtxt',
        'FL', 'Eq' , 'NotiCreateDate',
        'Div','plannergroup','MainWC','NotiSysStatus','OrderType',
        'failuremode', 'failuremech', 'cause', 'mtceact','mtceitem_subunit', 'mtceitem_part',
       'NotiType', 'FLobjType', 'FLClass', 'WorkSt', 'WorkFn',                                                                                                                                       
       'NotiCreateByID', 'NotiCreateBy', 'NotiCreateSec', 'NotiCompleteByID',
       'NotiCompleteBy', 'NotiCompleteSec', 'NotiUserStatus',  'complex', 'plant',
       'finished', 'canceled', 'dqp', 'needhist',  'Div']].to_excel(saved_dir+'DataQualityPassed.xlsx',index=False)
print(f'Time2Save DQP Excel File = {pd.Timestamp.now()-t1_start}')

Time2Save DQP Excel File = 0 days 00:00:18.691145


In [98]:
local_path = saved_dir+'DataQualityPassed.xlsx'
upload_largefile2sharepoint(file2upload=local_path)

NameError: name 'upload_largefile2sharepoint' is not defined

In [99]:
# gen excel all of sapsummary short report and failure codes
t1_start = pd.Timestamp.now()
sapsummary[(sapsummary.OrderType=='PM01')&(sapsummary.NotiType=='M1')&(sapsummary.finished==True)&(sapsummary.canceled==False)&(sapsummary.NotiCreateDate>='2019-06-01')] \
            .to_csv(saved_dir+'4FailureDataQualityCheck.csv',index=False)
print(f'Time2Save 4FailureDataQualityCheckCSV = {pd.Timestamp.now()-t1_start}')

Time2Save 4FailureDataQualityCheckCSV = 0 days 00:00:02.880305


In [100]:
local_path = saved_dir+'4FailureDataQualityCheck.csv'
upload_largefile2sharepoint(file2upload=local_path)

NameError: name 'upload_largefile2sharepoint' is not defined

In [101]:
saporder.columns

Index(['Order_Order', 'OrderDesc', 'MainActType', 'OrderCreateDate',
       'TecoDate', 'OrderSysStatus', 'Superior Order', 'ADDDATE',
       'OrderUserStatus', 'OrderBasicStart', 'OrderBasicFinish'],
      dtype='object')

In [102]:
#enfotech
t1_start = pd.Timestamp.now()
enfotech = sapsummary[['Noti', 'NotiDesc','notilongtxt',
                 'NotiCreateDate', 'NotiCreateYear','Complex', 'plant',
                 'FL','Eq','eqtype', 'Order', 'OrderType', 'OrderDesc',
                 'ActLaborCost', 'ActMatCost', 'ActExtCost','TotalCost',
                 'failuremode', 'failuremech', 'mtceact', 'mtceitem_subunit','mtceitem_part', 'cause',                 
                 'MainActType', 'OrderCreateDate','finished']]
enfotech = enfotech[(enfotech.OrderType<'PM09')&(sapsummary.NotiCreateYear>=2018)]
enfotech = enfotech.merge(enfo_saporder,left_on='Noti',right_on='Order_Noti',how='outer')
print(f'Time2Merge enfotech = {pd.Timestamp.now()-t1_start}')

Time2Merge enfotech = 0 days 00:03:27.899953


In [103]:
t1_start = pd.Timestamp.now()
enfotech['plant'] = enfotech.FL.str[:4].str.strip()
enfotech['order_plant'] = enfotech.Order_FL.str[:4].str.strip()
enfotech.to_excel(saved_dir+'EnfotechNotiOrder.xlsx', index = False)
print(f'Time2Save enfotech2Excel = {pd.Timestamp.now()-t1_start}')

Time2Save enfotech2Excel = 0 days 00:13:24.626802


In [104]:
baseurl = 'https://irpcplc.sharepoint.com'
basesite = '/sites/Seiya_Site' # every share point has a home.
docurl = '/Shared%20Documents/sap/Enfotech'
siteurl = baseurl + basesite 
target_url = basesite+docurl
ctx_auth = AuthenticationContext(siteurl)
ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)
ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.
target_folder = ctx.web.get_folder_by_server_relative_url(target_url)

local_path = saved_dir+'EnfotechNotiOrder.xlsx'
upload_largefile2sharepoint(file2upload=local_path)

baseurl = 'https://irpcplc.sharepoint.com'
basesite = '/sites/IRPCWebsite/PRO/EM' # every share point has a home.
docurl = '/Shared%20Documents/SAPPM_Python_Uploaded'
siteurl = baseurl + basesite 
target_url = basesite+docurl
ctx_auth = AuthenticationContext(siteurl)
ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)
ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.
target_folder = ctx.web.get_folder_by_server_relative_url(target_url)

An error occurred while retrieving token from XML response: AADSTS5000811: Unable to verify token signature. The signing key identifier does not match any valid registered keys.


ValueError: An error occurred while retrieving token from XML response: AADSTS5000811: Unable to verify token signature. The signing key identifier does not match any valid registered keys.

In [105]:
#order_wo_noti
t1_start = pd.Timestamp.now()
order_wo_noti = enfo_saporder.copy()[(enfo_saporder.Order_Noti.str.isspace())&((enfo_saporder.Order_SupOrder.isnull())|(enfo_saporder.Order_SupOrder.str.isspace()))]
order_wo_noti.loc[:,'Order_PlannerGroup'] = order_wo_noti.Order_MainWC.str[:3]
order_wo_noti = order_wo_noti.merge(pgdiv,left_on='Order_PlannerGroup',right_on='plannergroup',how='left')
order_wo_noti.loc[:,'Plant'] = order_wo_noti.Order_FL.str[:4].str.strip()
order_wo_noti = order_wo_noti.merge(complex,on='Plant', how='left')
order_wo_noti.to_excel(saved_dir+'OrderWithoutNoti.xlsx', index = False,engine='xlsxwriter')
print(f'Time2Save orderwithoutnoti = {pd.Timestamp.now()-t1_start}')

Time2Save orderwithoutnoti = 0 days 00:00:11.260163


In [106]:
local_path = saved_dir+'OrderWithoutNoti.xlsx'
upload_largefile2sharepoint(file2upload=local_path)

NameError: name 'upload_largefile2sharepoint' is not defined

In [107]:
sapsummary.columns

Index(['Complex', 'Plant', 'Plant4', 'Noti', 'NotiDesc', 'notilongtxt',
       'ref_pm_order', 'FL', 'fltype', 'FLabc', 'FLDesc', 'flstatus',
       'NotiDate', 'MalFnSt', 'MalFnEnd', 'ReqSt', 'ReqEn', 'NotiCreateDate',
       'NotiChanged', 'Eq', 'eqtype', 'eqclass', 'Eqabc', 'eqstatus', 'MainWC',
       'MtcePlant', 'PlantName', 'CCTR', 'BreakdownInd', 'NotiSysStatus',
       'NotiUserStatus', 'NotiPri', 'NotiPriTxt', 'Order', 'OrderDesc',
       'OrderType', 'NotiType', 'FLobjType', 'FLClass', 'WorkSt', 'WorkFn',
       'OrderBasicStart', 'OrderBasicFinish', 'ActLaborCost', 'ActMatCost',
       'ActExtCost', 'AmountOfWorkOrder', 'MH', 'NotiCreateYear', 'TTRMinute',
       'TotalCost', 'CashCost', 'notification', 'MainActType',
       'OrderCreateDate', 'TecoDate', 'OrderSysStatus', 'OrderUserStatus',
       'Superior Order', 'plannergroup', 'failuremode_codegroup',
       'failuemode_id', 'failuremode', 'cattype_failuremech',
       'failuremech_codegroup', 'failuremech_codegroupdes

In [108]:
t1_start = pd.Timestamp.now()
sapsummary.to_csv(saved_dir+'SapPM.csv',index=False)
print(f'Time2Save SAPPM.CSV = {pd.Timestamp.now()-t1_start}')

Time2Save SAPPM.CSV = 0 days 00:03:40.899627


In [109]:
local_path = saved_dir+'SapPM.csv'
upload_largefile2sharepoint(file2upload=local_path)

NameError: name 'upload_largefile2sharepoint' is not defined

In [ ]:
print(f'Time2Run import and merge SAP PM data from sql server = {pd.Timestamp.now()-t_start0}')
print(f'Finished on {pd.Timestamp.now()}')

In [ ]:
sapsummary.info()

In [ ]:
sapsummary

In [ ]:
# sapsummary[sapsummary.FCC==True]

In [ ]:
# xx = pd.read_csv('SAPPM.csv')

In [ ]:
# xx.info()

In [ ]:
%history